# Lab2- setCovering Problem with HillClimping Techniques

proceeding with the set covering problem but using Hill Climbing Techniques:

In [201]:
import random 
import logging
import collections

In [202]:
def problem(N, seed=None):
    random.seed(seed)
    array = [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]
    ranked_list = [(sorted(x),len(x)) for x in array]
    return ranked_list


## 1- Hill Climping: -

In [203]:
def better(old,new,discovered_Nums):
    if len(new)< len(old):
        # w is less for sure so we select the new node
        return True
    elif len(new) >= len(old):
        newCover = len(collections.Counter(new) - collections.Counter(discovered_Nums))
        oldCover = len(collections.Counter(old) - collections.Counter(discovered_Nums))
        if newCover > oldCover:
            return True
        else: 
            return False
    else:
        return False
    
def bestNode(betterNodes,discovered_Nums):
    best= betterNodes[0]
    for i in range(len(betterNodes)):
        if len(best)>= len(betterNodes[i]):
            bestCover = len(collections.Counter(best) - collections.Counter(discovered_Nums))
            nodeCover = len(collections.Counter(betterNodes[i]) - collections.Counter(discovered_Nums))
            if nodeCover >= bestCover:
                best = betterNodes[i]
        else:
            bestCover = len(collections.Counter(best) - collections.Counter(discovered_Nums))
            nodeCover = len(collections.Counter(betterNodes[i]) - collections.Counter(discovered_Nums))
            if nodeCover == bestCover:
                best = betterNodes[i]
    return best


In [207]:
def Tweak(state,nodes,discovered_Nums,N,mtype="fi"):
    # here i'm trying to select the perfect state to add and update the discovered_Nums so until now we are exploiting 
    stateNewNums = collections.Counter(state) - collections.Counter(discovered_Nums)
    # the stateNewNums are the nums that are not in the discovered nums yet
    stateOldNums = list((collections.Counter(state)- stateNewNums).keys())
    initstateOldNums = len(stateOldNums)
    nwState = state.copy()
    
    Nums = list(range(N))
    remNumstoCover_notAlready_Instate =list((collections.Counter(Nums) - collections.Counter(discovered_Nums)- collections.Counter(state)).keys())
    if len(remNumstoCover_notAlready_Instate)==0:
        return state
    rn = random.choice(remNumstoCover_notAlready_Instate)
    flag =1
    numtoRm= int()
    betterNodes = list()
    bestOfNodes = list()
    firstSolution= list()

    while len(remNumstoCover_notAlready_Instate) != 0:
        if initstateOldNums == 0 :
            # so here all the state numbers are new 
            nwState.append(rn)
            if tuple([sorted(nwState),len(nwState)]) in nodes:
                if (better(state,nwState,discovered_Nums)) & (mtype == "ss"):
                    # add to betterNodes
                    betterNodes.append(nwState)
                    nwState= state.copy()
                    remNumstoCover_notAlready_Instate.remove(rn)
                    if len(remNumstoCover_notAlready_Instate) == 0:
                    # here we didn't find any neighbour
                        break 
                    rn =random.choice(remNumstoCover_notAlready_Instate)
                    continue
                    
                elif (better(state,nwState,discovered_Nums)) & (mtype == "fi"):
                    #add to the first solution and return it
                    firstSolution = nwState
                    break
                    
            else:
                nwState= state.copy()
                remNumstoCover_notAlready_Instate.remove(rn)
                if len(remNumstoCover_notAlready_Instate) == 0:
                    # here we didn't find any neighbour
                    break 
                rn =random.choice(remNumstoCover_notAlready_Instate)
        else:
            if flag:
                numtoRm =random.choice(stateOldNums)
                stateOldNums.remove(numtoRm)
                nwState.remove(numtoRm)
                flag = 0
            
            nwState.append(rn)
            if tuple([sorted(nwState),len(nwState)]) in nodes:
                if (better(state,nwState,discovered_Nums))& (mtype == "ss"):
                    # add to betterNodes
                    betterNodes.append(nwState)
                    nwState= state.copy()
                    remNumstoCover_notAlready_Instate.remove(rn)
                    if len(remNumstoCover_notAlready_Instate) == 0:
                    # here we didn't find any neighbour that exist in our search problem
                        break 
                    rn =random.choice(remNumstoCover_notAlready_Instate)
                    continue
                    
                elif (better(state,nwState,discovered_Nums))& (mtype == "fi"):
                    #add to the first solution and return it
                    firstSolution = nwState
                    break
                
            else:
                nwState.remove(rn)
                remNumstoCover_notAlready_Instate.remove(rn)
                if len(remNumstoCover_notAlready_Instate) == 0:
                    flag =1
                    remNumstoCover_notAlready_Instate =list((collections.Counter(Nums) - collections.Counter(discovered_Nums)- collections.Counter(state)).keys())
                    if len(stateOldNums) == 0 :
                        break 
                rn =random.choice(remNumstoCover_notAlready_Instate)

    if len(betterNodes) !=0:
        bestOfNodes= bestNode(betterNodes,discovered_Nums)
    else:
        bestOfNodes = state

    if mtype=="fi":
        return firstSolution if len(firstSolution)!=0 else state
    else:
        return bestOfNodes    


In [ ]:
N = 100
nodes = problem(N,42)
print(nodes)
Tweak(state=[8,2,7,6],nodes=nodes,discovered_Nums=[8,2,6],N=N)

In [210]:
# first improvement => fi
# Steepest-step => ss
def Hill_Climping_Search(nodes,N,seed =42,mtype ="ss"):
      random.seed(seed)
      discovered_Nums=list()
      st_Index = random.randint(0, len(nodes)-1)
      st_node = nodes[st_Index][0]
      solution_nodes = list()
   
      for _ in range(len(nodes)):
            nwnode= Tweak(state=st_node,nodes=nodes,
                  discovered_Nums=discovered_Nums,
                  N=N,
                  mtype=mtype)
            discovered_Nums.extend(list((collections.Counter(nwnode)-collections.Counter(discovered_Nums)).keys()))
            solution_nodes.append(nwnode)
            
            if len(discovered_Nums) == N:
                  break
            
            nodes.remove(tuple([sorted(nwnode),len(nwnode)]))
            st_Index = random.randint(0, len(nodes)-1)
            st_node = nodes[st_Index][0]
      weight = sum(len(x) for x in solution_nodes)
      print(f"For N = {N} => the list has a weight W = {weight} and we used {len(solution_nodes)} nodes to cover it")
      

In [211]:
print("---------Hill Climping Algorithm------------") 
for N in [5, 10, 20, 100, 500, 1000]:
    nodes = problem(N,42)
    Hill_Climping_Search(nodes,N)

---------Hill Climping Algorithm------------
For N = 5 => the list has a weight W = 5 and we used 3 nodes to cover it
For N = 10 => the list has a weight W = 24 and we used 7 nodes to cover it
For N = 20 => the list has a weight W = 59 and we used 9 nodes to cover it
For N = 100 => the list has a weight W = 347 and we used 14 nodes to cover it
For N = 500 => the list has a weight W = 2991 and we used 20 nodes to cover it
For N = 1000 => the list has a weight W = 5797 and we used 19 nodes to cover it
